### Importación de datos



In [1]:
import pandas as pd

url1 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_1%20.csv"
url2 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_2.csv"
url3 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_3.csv"
url4 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_4.csv"

tienda1 = pd.read_csv(url1)
tienda2 = pd.read_csv(url2)
tienda3 = pd.read_csv(url3)
tienda4 = pd.read_csv(url4)

#0. Exploracion de datos

In [2]:
tienda1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2359 entries, 0 to 2358
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Producto                2359 non-null   object 
 1   Categoría del Producto  2359 non-null   object 
 2   Precio                  2359 non-null   float64
 3   Costo de envío          2359 non-null   float64
 4   Fecha de Compra         2359 non-null   object 
 5   Vendedor                2359 non-null   object 
 6   Lugar de Compra         2359 non-null   object 
 7   Calificación            2359 non-null   int64  
 8   Método de pago          2359 non-null   object 
 9   Cantidad de cuotas      2359 non-null   int64  
 10  lat                     2359 non-null   float64
 11  lon                     2359 non-null   float64
dtypes: float64(4), int64(2), object(6)
memory usage: 221.3+ KB


## df tienen mismas columnas

In [3]:

# def is_same_columns(all_data ):
#   set_col = set(all_data[0].columns)
#   set_col_type = set(all_data[0].dtypes)
#   for data in all_data:
#     if set(data.columns) != set_col:
#       return False
#     if set(data.dtypes) != set_col_type:
#       return False

#   return True


def is_same_columns(all_data):
    """
    Check if all DataFrames in a list have identical columns and dtypes.
    """
    if not all_data:
        return True  # Empty list — no conflict

    # Reference from first DataFrame
    ref = all_data[0]
    ref_cols = list(ref.columns)
    ref_dtypes = ref.dtypes.to_dict()

    for i, df in enumerate(all_data[1:], start=2):
        if not df.columns.equals(ref.columns):
            print(f"❌ Columns differ in DataFrame {i}")
            return False
        if not df.dtypes.equals(ref.dtypes):
            print(f"❌ Dtypes differ in DataFrame {i}")
            return False

    return True

all_data = [tienda1, tienda2, tienda3, tienda4]
is_same_columns(all_data)

True

## Ajuste formato tiempo

In [4]:
def adjust_date_format(df, column_name='Fecha de Compra'):
  df[column_name] = pd.to_datetime(df[column_name], errors='coerce', dayfirst=True)

#all_data = [tienda1, tienda2, tienda3, tienda4]
for df in all_data:
  adjust_date_format(df)


In [5]:
tienda1.head()

,Producto,Categoría del Producto,Precio,Costo de envío,Fecha de Compra,Vendedor,Lugar de Compra,Calificación,Método de pago,Cantidad de cuotas,lat,lon
0,Asistente virtual,Electrónicos,164300.0,6900.0,2021-01-16,Pedro Gomez,Bogotá,4,Tarjeta de crédito,8,4.60971,-74.08175
1,Mesa de comedor,Muebles,192300.0,8400.0,2022-05-18,Beatriz Morales,Medellín,1,Tarjeta de crédito,4,6.25184,-75.56359
2,Juego de mesa,Juguetes,209600.0,15900.0,2021-03-15,Juan Fernandez,Cartagena,1,Tarjeta de crédito,1,10.39972,-75.51444
3,Microondas,Electrodomésticos,757500.0,41000.0,2022-05-03,Juan Fernandez,Cali,4,Nequi,1,3.43722,-76.52250
4,Silla de oficina,Muebles,335200.0,20200.0,2020-11-07,Maria Alfonso,Medellín,5,Nequi,1,6.25184,-75.56359


## df tiene mismas Categoría del Producto

In [6]:
categorias = set(tienda1['Categoría del Producto'].unique())
print(categorias)

for name, df in zip(['tienda1', 'tienda2', 'tienda3', 'tienda4'], all_data):
  print(name,set(df['Categoría del Producto'].unique()) == categorias)

#categorias = set(tienda1['Categoría del Producto'].unique())

{'Electrónicos', 'Deportes y diversión', 'Muebles', 'Instrumentos musicales', 'Juguetes', 'Libros', 'Electrodomésticos', 'Artículos para el hogar'}
tienda1 True
tienda2 True
tienda3 True
tienda4 True


## revision global

In [7]:
#Using a specific category
df = tienda1

global_info ={}
for col in df.columns:
  if df[col].dtype == 'object':
    global_info[col] = df[col].unique()


#cat = 'Electrodomésticos'
#df[df['Categoría del Producto']=='Electrodomésticos']['Producto'].unique()

for k,v in global_info.items():
  print(k,v)



Producto ['Asistente virtual' 'Mesa de comedor' 'Juego de mesa' 'Microondas'
 'Silla de oficina' 'Bloques de construcción' 'Muñeca bebé' 'Lavavajillas'
 'Lavadora de ropa' 'Carrito de control remoto' 'Refrigerador'
 'Auriculares' 'Olla de presión' 'Bicicleta' 'Ciencia de datos con Python'
 'Balón de voleibol' 'Ajedrez de madera' 'TV LED UHD 4K' 'Vaso térmico'
 'Kit de bancas' 'Dashboards con Power BI' 'Cubertería' 'Set de ollas'
 'Batería' 'Cuerda para saltar' 'Estufa' 'Cama box' 'Impresora'
 'Mesa de noche' 'Armario' 'Dinosaurio Rex' 'Guitarra eléctrica'
 'Secadora de ropa' 'Sofá reclinable' 'Tablet ABXY' 'Cubo mágico 8x8'
 'Set de vasos' 'Modelado predictivo' 'Iphone 15' 'Mochila'
 'Balón de baloncesto' 'Auriculares con micrófono' 'Sillón'
 'Guitarra acústica' 'Iniciando en programación' 'Celular ABXY'
 'Smartwatch' 'Mesa de centro' 'Cama king' 'Smart TV' 'Pandereta']
Categoría del Producto ['Electrónicos' 'Muebles' 'Juguetes' 'Electrodomésticos'
 'Artículos para el hogar' 'Deportes 

## Productos por categoria

In [8]:
df = tienda1
productos_por_categoria = {}
for cat in df['Categoría del Producto'].unique():
  productos_por_categoria[cat] = df[df['Categoría del Producto']==cat]['Producto'].unique()
  print(cat,productos_por_categoria[cat])
  print()


Electrónicos ['Asistente virtual' 'Auriculares' 'TV LED UHD 4K' 'Impresora'
 'Tablet ABXY' 'Iphone 15' 'Auriculares con micrófono' 'Celular ABXY'
 'Smartwatch' 'Smart TV']

Muebles ['Mesa de comedor' 'Silla de oficina' 'Kit de bancas' 'Cama box'
 'Mesa de noche' 'Armario' 'Sofá reclinable' 'Sillón' 'Mesa de centro'
 'Cama king']

Juguetes ['Juego de mesa' 'Bloques de construcción' 'Muñeca bebé'
 'Carrito de control remoto' 'Ajedrez de madera' 'Dinosaurio Rex'
 'Cubo mágico 8x8']

Electrodomésticos ['Microondas' 'Lavavajillas' 'Lavadora de ropa' 'Refrigerador' 'Estufa'
 'Secadora de ropa']

Artículos para el hogar ['Olla de presión' 'Cubertería' 'Set de ollas' 'Set de vasos']

Deportes y diversión ['Bicicleta' 'Balón de voleibol' 'Vaso térmico' 'Cuerda para saltar'
 'Mochila' 'Balón de baloncesto']

Libros ['Ciencia de datos con Python' 'Dashboards con Power BI'
 'Modelado predictivo' 'Iniciando en programación']

Instrumentos musicales ['Batería' 'Guitarra eléctrica' 'Guitarra acústica

In [9]:
total_products = 0
for k,v in productos_por_categoria.items():
  total_products += len(v)
total_products

51

In [14]:
#all_data = [tienda1, tienda2, tienda3, tienda4]
# for df in all_data:
#   print(df.shape)
#   check_nan = df.isna().sum()
#   check_nan = check_nan[check_nan > 0]
#   print(check_nan)
#   print(df.describe().round(2))
#   print("-"*50)


def summarize_dataframes(dataframes, names=None,print_rev = False):
    """
    Print a concise summary for each DataFrame:
    - Shape (rows, columns)
    - Columns with missing values and their counts
    - Descriptive statistics (rounded to 2 decimals)
    """
    if names is None:
        names = [f"DataFrame {i+1}" for i in range(len(dataframes))]

    ans = {}
    for name, df in zip(names, dataframes):
        print(f"\n📊 Summary for {name}")
        print("=" * (12 + len(name)))

        # Shape
        print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")

        # Missing values
        na_counts = df.isna().sum()
        na_counts = na_counts[na_counts > 0]
        if not na_counts.empty:
            print("\nMissing values:")
            print(na_counts.sort_values(ascending=False))
        else:
            print("\nNo missing values ✅")

        # Descriptive statistics
        if print_rev:
          print("\nDescriptive statistics:")
          #print(df.describe(include='all').round(2))
          print(df.describe().round(2))
        print("-" * 60)

        ans[name] = df.describe().round(2)

    return ans
analisis = summarize_dataframes(all_data, ['tienda1', 'tienda2', 'tienda3', 'tienda4'])



📊 Summary for tienda1
Shape: 2359 rows × 12 columns

No missing values ✅
------------------------------------------------------------

📊 Summary for tienda2
Shape: 2359 rows × 12 columns

No missing values ✅
------------------------------------------------------------

📊 Summary for tienda3
Shape: 2359 rows × 12 columns

No missing values ✅
------------------------------------------------------------

📊 Summary for tienda4
Shape: 2358 rows × 12 columns

No missing values ✅
------------------------------------------------------------


In [17]:
analisis['tienda3']

,Precio,Costo de envío,Fecha de Compra,Calificación,Cantidad de cuotas,lat,lon
count,2359.00,2359.00,2359,2359.00,2359.00,2359.00,2359.00
mean,465459.77,24805.68,2021-08-22 16:01:25.459940608,4.05,2.97,5.44,-74.73
min,7600.00,0.00,2020-01-01 00:00:00,1.00,1.00,-4.22,-77.28
25%,57400.00,3100.00,2020-10-27 12:00:00,4.00,1.00,4.61,-75.56
50%,226300.00,11900.00,2021-09-06 00:00:00,5.00,2.00,4.61,-74.20
75%,666050.00,34950.00,2022-06-15 12:00:00,5.00,4.00,6.25,-74.08
max,2968200.00,158400.00,2023-03-31 00:00:00,5.00,24.00,11.54,-67.92
std,580643.39,30945.89,NaN,1.39,2.81,2.41,1.22


In [27]:
#just to check if there is a min less than cero
for name, df_analisis in analisis.items():
  print(name, df_analisis.loc['min','Precio'],df_analisis.loc['min','Costo de envío'])
  print()


tienda1 7600.0 0.0

tienda2 7600.0 0.0

tienda3 7600.0 0.0

tienda4 7600.0 0.0



In [12]:
import numpy as np

def review_outliers_iqr(df,colums_review = ['Precio','Costo de envío'],columns_filter=['Categoría del Producto','Producto'], threshold=1.5):
  """
  Detect potential outliers in numeric colums_review using the IQR method.
  but using columns filter to group

  note:It doesn’t necessarily mean that the distribution is perfectly clean —
  only that the IQR rule (with your chosen threshold) doesn’t flag any outliers.
  """
  ans = {}

  for col in colums_review:
    df_reveiw = df.groupby(columns_filter)[col].describe()
    df_reveiw.columns = [''.join(col).strip() for col in df_reveiw.columns.values] # Aplanar las columnas multiindex creadas en groupby
    df_reveiw = df_reveiw.reset_index()


    df_reveiw['lower_bound'] = df_reveiw['25%'] - (df_reveiw['75%']-df_reveiw['25%'])*threshold
    df_reveiw['upper_bound'] = df_reveiw['75%'] + (df_reveiw['75%']-df_reveiw['25%'])*threshold
    df_reveiw['outliers'] = ~((df_reveiw['upper_bound'] > df_reveiw['max']) | (df_reveiw['min'] > df_reveiw['lower_bound']))

    ans[col] = any(df_reveiw['outliers'])

  for col,outlier in ans.items():
      print(f"the column {col} {'possibly have' if outlier else 'doesn’t flag any'} outliers")

  return ans


review_outliers_iqr(df,colums_review = ['Precio','Costo de envío'],columns_filter=['Categoría del Producto','Producto'], threshold=1.5)



the column Precio doesn’t flag any outliers
the column Costo de envío doesn’t flag any outliers


{'Precio': False, 'Costo de envío': False}

In [13]:

for name, df in zip(['tienda1', 'tienda2', 'tienda3', 'tienda4'], all_data):
  print(name)
  review_outliers_iqr(df)


tienda1
the column Precio doesn’t flag any outliers
the column Costo de envío doesn’t flag any outliers
tienda2
the column Precio doesn’t flag any outliers
the column Costo de envío possibly have outliers
tienda3
the column Precio doesn’t flag any outliers
the column Costo de envío possibly have outliers
tienda4
the column Precio doesn’t flag any outliers
the column Costo de envío possibly have outliers


In [30]:
def review_outliers_2(df,column,columns_filter,threshold=1.5):
    """
    Detects outliers in `column` grouped by `columns_filter`
    using the IQR method.
    Returns a DataFrame containing only outlier rows.
    """

    # Describe stats per group
    df_review = df.groupby(columns_filter)[column].describe().reset_index()

    # Compute IQR and bounds
    iqr = df_review['75%'] - df_review['25%']
    df_review['lower_bound'] = df_review['25%'] - iqr * threshold
    df_review['upper_bound'] = df_review['75%'] + iqr * threshold

    # Merge back to original dataframe
    df_merged = df.merge(df_review[columns_filter + ['lower_bound', 'upper_bound']], on=columns_filter, how='left')

    # Filter rows that are outside bounds (outliers)
    df_outliers = df_merged[
        (df_merged[column] < df_merged['lower_bound']) |
        (df_merged[column] > df_merged['upper_bound'])
    ].copy()

    return df_outliers

#play with threshold=3
df_review = review_outliers_2(df, column = 'Costo de envío',columns_filter=['Categoría del Producto','Producto'], threshold=3)

df_review[['Producto', 'Categoría del Producto', 'Precio', 'Costo de envío','upper_bound']]


,Producto,Categoría del Producto,Precio,Costo de envío,upper_bound
103,Set de vasos,Artículos para el hogar,39800.0,8900.0,6025.0
117,Cuerda para saltar,Deportes y diversión,18000.0,7800.0,6000.0
131,Dinosaurio Rex,Juguetes,19200.0,7100.0,5100.0
228,Dinosaurio Rex,Juguetes,19300.0,5700.0,5100.0
310,Set de vasos,Artículos para el hogar,27700.0,10200.0,6025.0
372,Set de vasos,Artículos para el hogar,43500.0,8400.0,6025.0
540,Cuerda para saltar,Deportes y diversión,18300.0,7000.0,6000.0
606,Modelado predictivo,Libros,60300.0,11100.0,11050.0
657,Set de vasos,Artículos para el hogar,39500.0,8900.0,6025.0
696,Dinosaurio Rex,Juguetes,19800.0,9800.0,5100.0


In [31]:
df_review.shape

(19, 14)

#1. Análisis de facturación



# 2. Ventas por categoría

# 3. Calificación promedio de la tienda


# 4. Productos más y menos vendidos

# 5. Envío promedio por tienda

# Notas